# About
This notebook is used for generating artwworks using stylegan2

# Setup
- Install requirement

In [ ]:
!pip install tensorflow-gpu==1.15.0
!pip install h5py==2.10.0

     |████████████████████████████████| 411.5 MB 6.1 kB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 3.8 MB 44.7 MB/s 
     |████████████████████████████████| 503 kB 42.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=3bd1b84ca6555bc509e47b6aecb9c1dc924fa578c034afc5fd644438091c73d6
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

 - Prepare libary


> Using tar



In [ ]:
!gdown --id 1xQRmIyCU3y-Pc-uLqk0KWYi6_gDdQnuK
!pip install /content/stylegan2_dnnlib-0.1.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1sM7J0Pf-NE0KU9llxEyNjooVxIp4hURm
To: /content/stylegan2_dnnlib-0.1.tar.gz
100% 45.3k/45.3k [00:00<00:00, 34.5MB/s]
Processing ./stylegan2_dnnlib-0.1.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
DEPRECATION: Source distribution is being reinstalled despite an installed package having the same name and version as the installed package. pip 21.2 will remove support for this functionality. A possible replacement is use --force-reinstall. You can find discussion regarding this at https://github.com/pypa/pip/issues/8711.
  ERROR: Failed building wheel for stylegan2-dnnlib
Failed to build stylegan2-dnnlib
ERROR: Could not build wheels for stylegan2-dnnlib which use PEP 517 and cannot be installed directly




> Using wheel



In [ ]:
!gdown --id 1nuFaL8STEnBdiozB5ac5dZp3cwzqef05
!python -m pip install stylegan2_dnnlib-0.1-py3-none-any.whl

Downloading...
From: https://drive.google.com/uc?id=1yH9JeoISICdmYyOZFP16VSwKMTxSVkbp
To: /content/stylegan2_dnnlib-0.1-py3-none-any.whl
100% 53.5k/53.5k [00:00<00:00, 20.0MB/s]
Processing ./stylegan2_dnnlib-0.1-py3-none-any.whl


- Download pretrained weight



In [ ]:
!gdown --id 1IdQCUq5USG6TQ38HRLkeFkIsV0GUT09X

Downloading...
From: https://drive.google.com/uc?id=1IdQCUq5USG6TQ38HRLkeFkIsV0GUT09X
To: /content/stylegan2-cwka.pkl
100% 385M/385M [00:02<00:00, 189MB/s]


# Start creating
 - Import libaries and difine functions

In [ ]:
import dnnlib
import dnnlib.tflib as tflib
import pickle
import ipywidgets as widgets
import PIL.Image
import numpy as np

def load_networks(path):
    tflib.init_tf()
    with open(path, 'rb') as stream:
        G, D, Gs = pickle.load(stream, encoding='latin1')
    return G, D, Gs

def display_sample(artist, genre, style, seed, scale, truncation):
    batch_size = 1
    l1 = np.zeros((1,167))
    l1[0][artist] = 1
    l1[0][genre] = 1
    l1[0][style] = 1
    all_seeds = [seed] * batch_size
    all_z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:]) for seed in all_seeds]) # [minibatch, component]
    all_w = Gs.components.mapping.run(scale*all_z, np.tile(l1, (batch_size, 1))) # [minibatch, layer, component]
    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs)
    display(PIL.Image.fromarray(np.median(all_images, axis=0).astype(np.uint8)))

- Initialize instances

In [ ]:
pretrained_network_pkl = '/content/entropy_art_tools.pkl'
_, _, Gs = load_networks(pretrained_network_pkl)

Gs_syn_kwargs = dnnlib.EasyDict()
batch_size = 1
Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_syn_kwargs.minibatch_size = batch_size

artist = widgets.Dropdown(
    options=[('Unknown Artist', 0), ('Boris Kustodiev', 1), ('Camille Pissarro', 2), ('Childe Hassam', 3), ('Claude Monet', 4), ('Edgar Degas', 5), ('Eugene Boudin', 6), ('Gustave Dore', 7), ('Ilya Repin', 8), ('Ivan Aivazovsky', 9), ('Ivan Shishkin', 10), ('John Singer Sargent', 11), ('Marc Chagall', 12), ('Martiros Saryan', 13), ('Nicholas Roerich', 14), ('Pablo Picasso', 15), ('Paul Cezanne', 16), ('Pierre Auguste Renoir', 17), ('Pyotr Konchalovsky', 18), ('Raphael Kirchner', 19), ('Rembrandt', 20), ('Salvador Dali', 21), ('Vincent Van Gogh', 22), ('Hieronymus Bosch', 23), ('Leonardo Da Vinci', 24), ('Albrecht Durer', 25), ('Edouard Cortes', 26), ('Sam Francis', 27), ('Juan Gris', 28), ('Lucas Cranach The Elder', 29), ('Paul Gauguin', 30), ('Konstantin Makovsky', 31), ('Egon Schiele', 32), ('Thomas Eakins', 33), ('Gustave Moreau', 34), ('Francisco Goya', 35), ('Edvard Munch', 36), ('Henri Matisse', 37), ('Fra Angelico', 38), ('Maxime Maufra', 39), ('Jan Matejko', 40), ('Mstislav Dobuzhinsky', 41), ('Alfred Sisley', 42), ('Mary Cassatt', 43), ('Gustave Loiseau', 44), ('Fernando Botero', 45), ('Zinaida Serebriakova', 46), ('Georges Seurat', 47), ('Isaac Levitan', 48), ('Joaquã­n Sorolla', 49), ('Jacek Malczewski', 50), ('Berthe Morisot', 51), ('Andy Warhol', 52), ('Arkhip Kuindzhi', 53), ('Niko Pirosmani', 54), ('James Tissot', 55), ('Vasily Polenov', 56), ('Valentin Serov', 57), ('Pietro Perugino', 58), ('Pierre Bonnard', 59), ('Ferdinand Hodler', 60), ('Bartolome Esteban Murillo', 61), ('Giovanni Boldini', 62), ('Henri Martin', 63), ('Gustav Klimt', 64), ('Vasily Perov', 65), ('Odilon Redon', 66), ('Tintoretto', 67), ('Gene Davis', 68), ('Raphael', 69), ('John Henry Twachtman', 70), ('Henri De Toulouse Lautrec', 71), ('Antoine Blanchard', 72), ('David Burliuk', 73), ('Camille Corot', 74), ('Konstantin Korovin', 75), ('Ivan Bilibin', 76), ('Titian', 77), ('Maurice Prendergast', 78), ('Edouard Manet', 79), ('Peter Paul Rubens', 80), ('Aubrey Beardsley', 81), ('Paolo Veronese', 82), ('Joshua Reynolds', 83), ('Kuzma Petrov Vodkin', 84), ('Gustave Caillebotte', 85), ('Lucian Freud', 86), ('Michelangelo', 87), ('Dante Gabriel Rossetti', 88), ('Felix Vallotton', 89), ('Nikolay Bogdanov Belsky', 90), ('Georges Braque', 91), ('Vasily Surikov', 92), ('Fernand Leger', 93), ('Konstantin Somov', 94), ('Katsushika Hokusai', 95), ('Sir Lawrence Alma Tadema', 96), ('Vasily Vereshchagin', 97), ('Ernst Ludwig Kirchner', 98), ('Mikhail Vrubel', 99), ('Orest Kiprensky', 100), ('William Merritt Chase', 101), ('Aleksey Savrasov', 102), ('Hans Memling', 103), ('Amedeo Modigliani', 104), ('Ivan Kramskoy', 105), ('Utagawa Kuniyoshi', 106), ('Gustave Courbet', 107), ('William Turner', 108), ('Theo Van Rysselberghe', 109), ('Joseph Wright', 110), ('Edward Burne Jones', 111), ('Koloman Moser', 112), ('Viktor Vasnetsov', 113), ('Anthony Van Dyck', 114), ('Raoul Dufy', 115), ('Frans Hals', 116), ('Hans Holbein The Younger', 117), ('Ilya Mashkov', 118), ('Henri Fantin Latour', 119), ('M.C. Escher', 120), ('El Greco', 121), ('Mikalojus Ciurlionis', 122), ('James Mcneill Whistler', 123), ('Karl Bryullov', 124), ('Jacob Jordaens', 125), ('Thomas Gainsborough', 126), ('Eugene Delacroix', 127), ('Canaletto', 128)],
    value=4,
    description='Artist: '
)
genre = widgets.Dropdown(
    options=[('Abstract Painting', 129), ('Cityscape', 130), ('Genre Painting', 131), ('Illustration', 132), ('Landscape', 133), ('Nude Painting', 134), ('Portrait', 135), ('Religious Painting', 136), ('Sketch And Study', 137), ('Still Life', 138), ('Unknown Genre', 139)],
    value=135,
    description='Genre: '
)
style = widgets.Dropdown(
    options=[('Abstract Expressionism', 140), ('Action Painting', 141), ('Analytical Cubism', 142), ('Art Nouveau', 143), ('Baroque', 144), ('Color Field Painting', 145), ('Contemporary Realism', 146), ('Cubism', 147), ('Early Renaissance', 148), ('Expressionism', 149), ('Fauvism', 150), ('High Renaissance', 151), ('Impressionism', 152), ('Mannerism Late Renaissance', 153), ('Minimalism', 154), ('Naive Art Primitivism', 155), ('New Realism', 156), ('Northern Renaissance', 157), ('Pointillism', 158), ('Pop Art', 159), ('Post Impressionism', 160), ('Realism', 161), ('Rococo', 162), ('Romanticism', 163), ('Symbolism', 164), ('Synthetic Cubism', 165), ('Ukiyo-e', 166)],
    value=140,
    description='Style: '
)
seed = widgets.IntSlider(min=0, max=100000, step=1, value=0, description='Seed: ')
scale = widgets.FloatSlider(min=0, max=5, step=0.01, value=1, description='Scale: ')
truncation = widgets.FloatSlider(min=-2, max=2, step=0.1, value=1, description='Truncation: ')
top_box = widgets.HBox([artist, genre, style])
bot_box = widgets.HBox([seed, scale, truncation])
ui = widgets.VBox([top_box, bot_box])

- Start generates

In [ ]:
out = widgets.interactive_output(display_sample, {'artist': artist, 'genre': genre, 'style': style, 'seed': seed, 'scale': scale, 'truncation': truncation})

display(ui, out)

Output()